In [1]:
include("./Bilevel/Bilevel.jl")
include("utils.jl")
using .Bilevel
using Plots
using ArgParse, PrettyTables

In [ ]:
# -- DEFAULT PARAMETERS OF A SIMULATION --
verbose     = true

# ------- Choice of the model -------
bilevel     = false 
storage     = false 

# ------- Profiles parameters -------
nb_days     = 2 
delta_t     = 60 
peak_power  = 7.0 
EV          = false 
EHP         = false 

# ------- Users parameters -------
PV_pen        = 1.0     
storage_pen   = 1.0   
storage_eff   = 0.98
storage_cost  = 500.0   # kEUR/MVA
amort_storage = 15      # years
PV_capa       = 0.4     # max in MVA
PV_cost       = 500.0   # EUR/kWc 
PV_conv_cost  = 200.0   # kEUR/MVA
amort_PV_conv = 10      # years
amort_PV      = 25      # years 
EIC           = 0.3     # EUR/kWh  
EEC           = 0.1     # EUR/kWh   
DSOEC         = 0.1     # EUR/kWh  
GCC           = 80.0    # EUR/kVA_year
cos_phi       = 0.95  

# ------- DSO parameters -------
substation_cost   = 1e3  # kEUR/MVA
amort_DSO         = 50  
interest_rate_DSO = 0.06 
money_basis = 1.0
weight_I = weight_V = 1.0e-2
weight_obj1 = weight_obj2 = 1.0

In [5]:
include("main_bilevel.jl")
network.buses


#--------------------------------------------------------------------------------
#                                                                                
#                                  Bilevel DNEP                                   
#                                                                                
#                                Graduation work                                 
#                                                                                
#--------------------------------------------------------------------------------
# @ Manon Cornet

Running a simulation with the following characteristics:
--------------------------------------------------------

╭─────────────┬───────┬───────┬─────────┬─────────┬─────────┬─────────┬─────────┬───────────╮
│  Model Type │    EV │   EHP │ Delta_t │ PV_CAPA │     EIC │     EEC │   DSOEC │       GCC │
│         [-] │   [-] │   [-] │   [min] │   [MVA] │ [€/kWh] │ [€/kWh] │ [€/kWh] │ [€/kVA/y] │
├─────────

4-element Vector{Main.Bilevel.Bus}:
 Main.Bilevel.Substation(Main.Bilevel.Node(1, (x = 0.0, y = 0.1)), false, (V_min = 0.95, V_max = 1.05), 15.0, 0.0, nothing, nothing, nothing)
 Main.Bilevel.Substation(Main.Bilevel.Node(2, (x = 0.0, y = 0.0)), false, (V_min = 0.95, V_max = 1.05), 15.0, 0.0, nothing, nothing, nothing)
 Main.Bilevel.User(Main.Bilevel.Node(3, (x = 0.1, y = 0.1)), (V_min = 0.95, V_max = 1.05), 0.4, nothing, Main.Bilevel.Profile([0.86704535386329, 0.6751924683412124, 0.24981686633378278, 1.5789764924404954], 720), Main.Bilevel.PV(Main.Bilevel.Profile([1.0, 0.9190765189761652, 0.1, 0.09190765189761652], 720), (max_q = 0.3, slope = -1.0), nothing, nothing, nothing), Storage(0.98, nothing, nothing, nothing), 0.9)
 Main.Bilevel.User(Main.Bilevel.Node(4, (x = 0.2, y = 0.1)), (V_min = 0.95, V_max = 1.05), 0.4, nothing, Main.Bilevel.Profile([0.28666628695107366, 0.24837434261737962, 0.46309441311790805, 0.7650165650927456], 720), nothing, nothing, 0.9)

In [6]:
model = Bilevel.build_model(simulation; set_names=true)
simulation.storage

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-10
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1


[ Info: Building model...
[ Info: Built model in 0.01 seconds


true

In [7]:
return_value = Bilevel.solve_model(model, formulation.powerflow)
save_struct(network_topology, "network_topology.json")
save_struct(network, "network_data.json")
using JuMP
value.(model[:storage_capacity])

Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1
Set parameter TimeLimit to value 600
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 474 rows, 333 columns and 1772 nonzeros
Model fingerprint: 0x0469c5cf
Model has 20 quadratic constraints
Variable types: 316 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [1e-09, 2e+05]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e-02, 1e+00]
  Bounds range     [4e-01, 1e+00]
  RHS range        [1e-01, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 204 rows and 95 columns
Presolve time: 0.00s
Presolved: 270 rows, 238 columns, 1194 nonzeros
Presolved model has 20 quadratic constraint(s)
Variable types: 224 continuous, 14 integer (14 binary)

Root relaxation: objective 3.6

2-element Vector{Float64}:
 0.0
 0.0

In [ ]:
thesis_dir = "/Users/manoncornet/Documents/University/TFE/ThesisWriting/Master_Thesis/figures" 
for t in 1:model[:time_steps]
    Bilevel.print_network_tikz(model[:network_data], t, 40, 40; dir=thesis_dir, filename="OneLevel_graph_$(t)")
end